In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import env

In [4]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

- DATA Wrangling aquireing the data 

In [5]:
sql = '''Select bathroomcnt, bedroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips from properties_2017
join propertylandusetype using (propertylandusetypeid)
join predictions_2017 using (parcelid)
where propertylandusedesc = "Single Family Residential"
AND predictions_2017.transactiondate like '2017%%'
'''

df = pd.read_sql(sql, get_connection("zillow"))
df.head()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,2.5,2.0,1348.0,343967.0,1979.0,3628.14,6059.0
1,2.0,3.0,1221.0,367034.0,1955.0,4589.60,6059.0
2,2.0,3.0,1462.0,79713.0,1950.0,1269.65,6037.0
3,1.0,2.0,768.0,270458.0,1926.0,3373.09,6037.0
4,4.0,4.0,3696.0,940251.0,1989.0,10313.18,6037.0


In [6]:
df.shape

(52441, 7)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52441 entries, 0 to 52440
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   52441 non-null  float64
 1   bedroomcnt                    52441 non-null  float64
 2   calculatedfinishedsquarefeet  52359 non-null  float64
 3   taxvaluedollarcnt             52440 non-null  float64
 4   yearbuilt                     52325 non-null  float64
 5   taxamount                     52437 non-null  float64
 6   fips                          52441 non-null  float64
dtypes: float64(7)
memory usage: 2.8 MB


In [8]:
df.describe()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
count,52441.000000,52441.000000,52359.000000,5.244000e+04,52325.000000,52437.000000,52441.000000
mean,2.299403,3.300681,1922.890754,5.296882e+05,1963.393884,6453.511209,6049.132149
std,1.022772,0.949094,1004.365838,7.518946e+05,23.133571,8755.568452,21.029067
min,0.000000,0.000000,128.000000,1.000000e+03,1878.000000,49.180000,6037.000000
25%,2.000000,3.000000,1268.000000,1.937475e+05,1950.000000,2656.220000,6037.000000
50%,2.000000,3.000000,1659.000000,3.736120e+05,1961.000000,4647.740000,6037.000000
75%,3.000000,4.000000,2306.000000,6.193012e+05,1979.000000,7377.500000,6059.000000
max,18.000000,14.000000,21929.000000,4.906124e+07,2016.000000,586639.300000,6111.000000


In [9]:
df.isnull().sum()

bathroomcnt                       0
bedroomcnt                        0
calculatedfinishedsquarefeet     82
taxvaluedollarcnt                 1
yearbuilt                       116
taxamount                         4
fips                              0
dtype: int64

In [10]:
df.columns[df.isnull().any()]

Index(['calculatedfinishedsquarefeet', 'taxvaluedollarcnt', 'yearbuilt',
       'taxamount'],
      dtype='object')

In [11]:
df = df.dropna()

- there is 52k people in this dataset i think losing like 200 is okay :)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52315 entries, 0 to 52440
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   52315 non-null  float64
 1   bedroomcnt                    52315 non-null  float64
 2   calculatedfinishedsquarefeet  52315 non-null  float64
 3   taxvaluedollarcnt             52315 non-null  float64
 4   yearbuilt                     52315 non-null  float64
 5   taxamount                     52315 non-null  float64
 6   fips                          52315 non-null  float64
dtypes: float64(7)
memory usage: 3.2 MB


In [13]:
df = df.astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52315 entries, 0 to 52440
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype
---  ------                        --------------  -----
 0   bathroomcnt                   52315 non-null  int64
 1   bedroomcnt                    52315 non-null  int64
 2   calculatedfinishedsquarefeet  52315 non-null  int64
 3   taxvaluedollarcnt             52315 non-null  int64
 4   yearbuilt                     52315 non-null  int64
 5   taxamount                     52315 non-null  int64
 6   fips                          52315 non-null  int64
dtypes: int64(7)
memory usage: 3.2 MB


In [ ]:
plt.figure(figsize=(16, 3))

# List of columns
cols = ['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt']

for i, col in enumerate(cols):
    
    # i starts at 0, but plot nos should start at 1
    plot_number = i + 1 
    
    # Create subplot.
    plt.subplot(1,4, plot_number)
    
    # Title with column name.
    plt.title(col)
    
    # Display histogram for column.
    df[col].hist(bins=5, edgecolor='black')
    
    # Hide gridlines.
    plt.grid(False)

In [14]:
df.fips.value_counts()

6037    33875
6059    14082
6111     4358
Name: fips, dtype: int64

In [ ]:
df['County'] = df.fips.map({6037:'LA', 6059:'Orange', 6111:'Ventura'})

df.head()

In [18]:
def fix_fips(df):
    df['County'] = df.fips.map({6037:'LA', 6059:'Orange', 6111:'Ventura'})
    return df


In [16]:
fix_fips(df)

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,County
0,2,2,1348,343967,1979,3628,6059,Orange
1,2,3,1221,367034,1955,4589,6059,Orange
2,2,3,1462,79713,1950,1269,6037,LA
3,1,2,768,270458,1926,3373,6037,LA
4,4,4,3696,940251,1989,10313,6037,LA
...,...,...,...,...,...,...,...,...
52436,3,3,1874,1055000,1964,12068,6037,LA
52437,1,3,876,148393,1913,1866,6037,LA
52438,6,3,4892,3984381,1953,48135,6037,LA
52439,2,4,2544,149688,1925,1802,6037,LA


In [17]:
for column in df.columns:
    print(column)
    print(df[column].value_counts())
    print("-------------------------------")

bathroomcnt
2     25818
3     11577
1     10392
4      2909
5      1027
6       369
7       104
8        55
0        41
9        13
10        5
11        3
18        1
13        1
Name: bathroomcnt, dtype: int64
-------------------------------
bedroomcnt
3     23346
4     15226
2      8333
5      3970
6       633
1       605
7       106
0        57
8        24
9         8
12        3
10        2
14        1
11        1
Name: bedroomcnt, dtype: int64
-------------------------------
calculatedfinishedsquarefeet
1120    128
1200    119
1080    102
1040     86
936      86
       ... 
5781      1
4573      1
5683      1
5707      1
4660      1
Name: calculatedfinishedsquarefeet, Length: 4720, dtype: int64
-------------------------------
taxvaluedollarcnt
455000     34
600000     30
450000     24
500000     24
550000     24
           ..
4112590     1
797358      1
379955      1
1421827     1
2597425     1
Name: taxvaluedollarcnt, Length: 38846, dtype: int64
-------------------------------
y

In [ ]:
df.taxamount.

In [ ]:
the wrangling of the data in the kalpana dataset is good too go all i would need to do is add the edits to aquire the data, 

In [ ]:
def handle_outliers(df):
    """Manually handle outliers that do not represent properties likely for 99% of buyers and zillow visitors"""
    df = df[df.bathroomcnt <= 6]
    
    df = df[df.bedroomcnt <= 6]

    df = df[df.taxvaluedollarcnt < 2_000_000]

    return df